# Reinforcement Learning baseline in Python with stable-baselines3
This code will make your life easier if you want to try Reinforcement Learning (RL) as a solution to kaggle's kore 2022 challenge.
One of the (multiple) difficulties of RL is achieving a clean implementation. While you can of course try to build yourself
one of the RL models described in literature, chances are that you will spend more time debugging your model than actually competing.

[stable-baselines3](https://stable-baselines3.readthedocs.io/en/master/#) is powerful RL library with a number of very nice features for this competition:
- It implements the most popular modern deep RL algorithms
- It is simple and ellegant to use
- It is rather well documented
- There are plenty of tutorials and examples

In other words, it's a fantastic starting point. Alas, it requires an environment compatible with OpenAI-gym and the kore environment is not. What you'll find in this notebook is **KoreGymEnv**, a wrapper around the kore env that makes it play nice with stable-baselines3. It includes very simple feature and action engineering, so the only thing you need to care about is building upon them, choosing a model and throwing yourself into the cold, unforgiving and yet very rewarding reinforcement learning waters ;)

As a bonus, this notebook also demonstrates the end-to-end process that you need to follow to submit any model with external dependencies. Click on submit and you're good to go.

#### Notes:

- In stable-baselines3, states and actions are numpy arrays. In the kore environment, states are lists of dicts and actions are dicts with shipyard ids as keys and shipyard actions as values. Thus, we need an interface to "translate" them. This interface is effectively where you implement your state & action engineering. You'll find more details in the KoreGymEnv class.
- In the ideal case, you would use self-play and let your agent play a very large number of games against itself, improving at ever step. Unfortunately, [it is not clear how to implement self-play in the kore env](https://www.kaggle.com/competitions/kore-2022/discussion/323382). So we have to train against static opponents. In this baseline, we'll use the starter bot. Of course, nothing prevents you from implementing pseudo-self-play and train against ever improving versions of your agent.

In [1]:
# Train a PPO agent
from environment import KoreGymEnv
from stable_baselines3 import PPO

kore_env = KoreGymEnv()
model = PPO('MlpPolicy', kore_env, verbose=1)
model.learn(total_timesteps=100000)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 399      |
|    ep_rew_mean     | 1.3e+03  |
| time/              |          |
|    fps             | 20       |
|    iterations      | 1        |
|    time_elapsed    | 100      |
|    total_timesteps | 2048     |
---------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 399          |
|    ep_rew_mean          | 1.33e+03     |
| time/                   |              |
|    fps                  | 22           |
|    iterations           | 2            |
|    time_elapsed         | 178          |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 0.0010416467 |
|    clip_fraction        | 0.0108       |
|    clip_range           | 0.2          |
|    en

KeyboardInterrupt: 

In [3]:
import os
import sys
KAGGLE_AGENT_PATH = "/kaggle_simulations/agent/"
if os.path.exists(KAGGLE_AGENT_PATH):
    sys.path.insert(0, os.path.join(KAGGLE_AGENT_PATH, 'lib'))
else:
    sys.path.insert(0, os.path.join(os.getcwd(), 'lib'))
    centre.multimedia@ca-cmds.fr

In [2]:
pickle.format_version

'4.0'

# Utils

### Check that we have a valid environment

In [2]:
# The bad news: this check will fail in the kaggle docker environment. The most likely reason is a version mismatch between packages.
# The good news: That's alright since everything else works! We're doing some unconventional dependency management here, so we'll have to live with
# a failed check.

from stable_baselines3.common.env_checker import check_env
from environment import KoreGymEnv

env = KoreGymEnv()
check_env(env)

1
0
2
1
3
2
4
3
5
4
6
5
7
6
8
7
9
8
10
9
11
10


# Train the agent!

In [8]:
from stable_baselines3 import PPO
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.monitor import Monitor
from environment import KoreGymEnv

In [9]:
kore_env = KoreGymEnv(config=dict(randomSeed=997269658))  # TODO: This seed is not enough. Seed everything!
monitored_env = Monitor(env=kore_env)
model = PPO('MlpPolicy', monitored_env, verbose=1)

Using cpu device
Wrapping the env in a DummyVecEnv.


In [11]:
%%time
# For serious training, likely many more iterations will be needed, as well as hyperparameter tuning!
# Even so, sometimes training will still fail. RL is like that. Try a couple times with the same config before giving up!
model.learn(total_timesteps=5000)  

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 399      |
|    ep_rew_mean     | 800      |
| time/              |          |
|    fps             | 26       |
|    iterations      | 1        |
|    time_elapsed    | 77       |
|    total_timesteps | 2048     |
---------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 399          |
|    ep_rew_mean          | 303          |
| time/                   |              |
|    fps                  | 26           |
|    iterations           | 2            |
|    time_elapsed         | 156          |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 0.0025211885 |
|    clip_fraction        | 0.0287       |
|    clip_range           | 0.2          |
|    entropy_loss         | -2.83        |
|    explained_variance   | 0.000203     |
|    learning_r

In [12]:
# Watch it mercilessly beat the baseline bot - Note: The current episode might not be over yet
kore_env.render(mode="ipython", width=1000, height=800)

In [13]:
model.save("baseline_agent")

# Evaluate agent performance

In [14]:
import numpy as np

eval_env = KoreGymEnv(config=dict(strict=True))  # The 'strict' flags sets rewards to 1 if the agent won the episode and 0 else. Useful for evaluation.
monitored_env = Monitor(env=eval_env)
model_loaded = PPO.load('baseline_agent')

def evaluate(model, num_episodes=1):
    """
    Evaluate a RL agent - Adapted from 
    https://colab.research.google.com/github/Stable-Baselines-Team/rl-colab-notebooks/blob/sb3/stable_baselines_getting_started.ipynb
    :param model: (BaseRLModel object) the RL Agent
    :param num_episodes: (int) number of episodes to evaluate it
    :return: (float) Mean reward for the last num_episodes
    """
    all_episode_rewards = []
    for i in range(num_episodes):
        episode_rewards = []
        done = False
        obs = monitored_env.reset()
        while not done:
            action, _ = model.predict(obs)
            obs, _, done, info = monitored_env.step(action)
            if done:
                agent_reward = monitored_env.env.raw_obs.players[0][0]
                opponent_reward = monitored_env.env.raw_obs.players[1][0]
                reward = agent_reward > opponent_reward
            else:
                reward = 0
            # print(reward)
            # monitored_env.render(mode='ipython', height=400, width=300)
            episode_rewards.append(reward)

        all_episode_rewards.append(sum(episode_rewards))

    mean_episode_reward = np.mean(all_episode_rewards)
    print("Mean reward:", mean_episode_reward, "Num episodes:", num_episodes)

    return mean_episode_reward

evaluate(model_loaded, 3)

Mean reward: 0.3333333333333333 Num episodes: 3


0.3333333333333333

### Prepare the submission

In [15]:
%%capture
# This is for debugging purposes only before submitting - Are there any errors?
from kaggle_environments import make
from config import OPPONENT
env = make("kore_fleets", debug=True)
env.run(['main.py', OPPONENT])

In [16]:
!tar -czf submission.tar.gz main.py config.py environment.py reward_utils.py baseline_agent.zip lib